In [1]:
import pandas as pd
import numpy as np
import re
import os
import logging

In [39]:
df = pd.read_csv("db_init/data/Books.csv", sep=",", encoding="UTF-8")
dfr = pd.read_csv("db_init/data/Ratings.csv", sep=",", encoding="UTF-8")
dfu = pd.read_csv("db_init/data/Users.csv", sep=",", encoding="UTF-8")

C:\Users\tomas\AppData\Local\Temp\ipykernel_10124\2638243751.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("db_init/data/Books.csv", sep=",", encoding="UTF-8")


In [2]:
logging.warning("Remain calm!")

In [3]:
directory_name = "mummy"
os.mkdir(directory_name)

In [4]:
def fix_special_characters(text):

    # Mapa pro opravy specifických znaků
    char_map = {
        # Běžné Latin-1/CP1252 znaky čtené jako UTF-8
        'Ã©': 'é',
        'Ã¨': 'è',
        'Ã­': 'í',
        'Ã§': 'ç',
        'Ã³': 'ó',
        'Ã¡': 'á',
        'Ã´': 'ô',
        'Ã¼': 'ü',
        'Ã¶': 'ö',
        'Ã½': 'ý',
        'Ã£': 'ã',
        'Ãº': 'ú',

        # Běžné Latin-1/CP1252 uppercase a ß čtené jako UTF-8 (často se zobrazí jako Ã? + znak)
        'Ã?Â¤': 'ä', 
        'Ã?Â©': 'é', 
        'Ã?Â¼': 'ü', 
        'Ã?Â¶': 'ö', 
        'Ã?Â?': 'Ü', 
        'Ã?Â?': 'Ä', 
        'Ã?Â?': 'Ö', 
        'Ã?Â?': 'É', 
        'Ã?Â?': 'Á', 
        'Ã?Â?': 'Í', 
        'Ã?ÂŸ': 'ß', 

        # Další běžné mojibake vzory pro španělštinu
        'Â¿': '¿', 
        'Â¡': '¡',

        # České znaky
        'ÄŤ': 'č',
        'Å™': 'ř',
        'Å¡': 'š',
        'Å¾': 'ž',
        'Ãº': 'ú', 

        # Příklad opravy pro 'Ã? sucesso solo 50 anni fa' - vypadá to, že 'Ã? ' by mělo být 'Il '
        'Ã? ': 'Il ',
    }

    if isinstance(text, str):  # Zkontroluj, že text je řetězec
        for wrong, correct in char_map.items():
            text = text.replace(wrong, correct)
    return text

In [5]:
def vector(df):
    # Mapa pro opravy specifických znaků
    char_map = {
        # Běžné Latin-1/CP1252 znaky čtené jako UTF-8
        'Ã©': 'é',
        'Ã¨': 'è',
        'Ã­': 'í',
        'Ã§': 'ç',
        'Ã³': 'ó',
        'Ã¡': 'á',
        'Ã´': 'ô',
        'Ã¼': 'ü',
        'Ã¶': 'ö',
        'Ã½': 'ý',
        'Ã£': 'ã',
        'Ãº': 'ú',
        # Běžné Latin-1/CP1252 uppercase a ß čtené jako UTF-8 (často se zobrazí jako Ã? + znak)
        'Ã?Â¤': 'ä', 
        'Ã?Â©': 'é', 
        'Ã?Â¼': 'ü', 
        'Ã?Â¶': 'ö', 
        'Ã?Â?': 'Ü', 
        'Ã?Â?': 'Ä', 
        'Ã?Â?': 'Ö', 
        'Ã?Â?': 'É', 
        'Ã?Â?': 'Á', 
        'Ã?Â?': 'Í', 
        'Ã?ÂŸ': 'ß', 
        # Další běžné mojibake vzory pro španělštinu
        'Â¿': '¿', 
        'Â¡': '¡',
        # České znaky
        'ÄŤ': 'č',
        'Å™': 'ř',
        'Å¡': 'š',
        'Å¾': 'ž',
        'Ãº': 'ú', 
        # Příklad opravy pro 'Ã? sucesso solo 50 anni fa' - vypadá to, že 'Ã? ' by mělo být 'Il '
        'Ã? ': 'Il ',
    }

    # Pracuj jen se sloupci typu object (text)
    text_cols = df.select_dtypes(include="object").columns

    for col in text_cols:
        for wrong, correct in char_map.items():
            # Vektorizovaná náhrada znaků
            df[col] = df[col].str.replace(wrong, correct, regex=False)
    return df

In [34]:
def remove_special_characters_numpy(df):
    pattern = r'[\\"&amp;^"$]'
    cols = df.select_dtypes(include="object").columns

    for col in cols:
        values = df[col].values.astype(str)  # convert to NumPy array of strings
        mask = pd.Series(values).str.contains(pattern, regex=True).values  # boolean mask
        cleaned = pd.Series(values).str.replace(pattern, '', regex=True).str.strip().values
        df[col] = np.where(mask, cleaned, values)

    return df

In [8]:
%%timeit
df2 = fix_special_characters(df)

1.16 μs ± 10.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [9]:
%%timeit
df2 = vector(df)

7 s ± 24.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
%%timeit
df3 = remove_special_characters_numpy(df)

2.74 s ± 120 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def remove_characters(df):   
    columns_to_change = df.select_dtypes(include=["object"]).columns
    df = df[columns_to_change].contains()

    for col in columns_to_change:
        df[col] = df[col].str.replace(r'[\\"&amp;^"$]', '"', regex=True).str.strip('"')
    
    return df

In [ ]:
def remove_characters_vec(df):   
    columns_to_change = df.select_dtypes(include=["object"]).columns
    #mask_specail_char = columns_to_change.str.contains(r'\\"|\\|&amp;|^"|"$', regex=True)
    # Vybereme všechny sloupce s textovými hodnotami
    columns_to_change = df.select_dtypes(include=["object"]).columns
    
    # Odstraníme speciální znaky z textových sloupců
    for col in columns_to_change:
        # Použijeme regex pro odstranění těchto znaků
        df[col] = df[col].str.replace(r'[\\"&amp;^"$]', '', regex=True).str.strip()

    return df

In [33]:
%%timeit
df2 = remove_characters(df)

2 s ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit
df2_vec = remove_characters(df)

1.95 s ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
def text_pre(df):
    text_cols = df.select_dtypes(include=["object"]).columns
    text_cols = text_cols[~text_cols.str.contains("ISBN|URL", regex=True)]

    # Převede jen vybrané sloupce do NumPy array
    arr = df[text_cols].to_numpy()

    # Funkce pro čištění (vektorizovaná)
    vectorized_clean = np.vectorize(lambda x: re.sub(r'[!@#$%^]', '', x))

    # Aplikuj na celou matici
    cleaned_arr = vectorized_clean(arr)

    # Vrať zpět do DataFrame
    df[text_cols] = cleaned_arr

    return df

In [42]:
%%timeit
df4 = text_pre(df)

TypeError: expected string or bytes-like object, got 'float'